In this project you are in the shoes of a computational neuroscientist who has just developed a new analysis tool. In this hypothetical, you have just developed an approach for Bayesian decoding of stimulus values from observed neural recordings. However, since you are the first to ever apply this technique, you need to validate that it is working. For this project, you will simulate synthetic neural activity for which you know the ground truth encoding model with respect to a stimulus. This is good practice for anytime you are developing new models or methods of analysis, to ensure not just that they work but that they actually address the scientific question at hand, they should be validated on data with a known ground truth.

You will be performing Bayesian decoding as described in [this paper](https://journals.physiology.org/doi/full/10.1152/jn.1998.79.2.1017#_i24) in the section titled "PROBABILISTIC APPROACH: BAYESIAN METHODS" (you have to yell it). First, spend some time reading the section's text, allowing yourself to look up any terms you may not recognize, or ask your peers or instructors questions. One of the primary purposes of this exercise is for you to gain practice ensuring your own deep understanding of published work. One of the best ways for you to efficiently gain deep understanding is to have someone who already put in the work building intuition explain it to you in terms of what you already know.

I will briefly describe the method now, but again please read the text before reading this primer.






I will be adopting the variable usage from the linked paper for consistency. If you recall from our discussions of encoding models using GLMs, an encoding model seeks to estimate $P(n|x)$, where $n$ is the observed neural response and $x$ is some stimulus value. The example in the paper for $x$ is an animal's physical location, and $x$ is therefore taken to be 2-dimensional (x,y). $P(n|x)$ would be some set of parameters that describe the probability of firing n spikes at position $x$. For a GLM, this would be a filter that is multiplied by your position (or history of positions), and then passed through a nonlinearity to obtain firing rates. However, it is important to recognize than an encoding model can be **anything** that gives you a probability of observing spikes. An encoding model can be the number one, where neuron just spike, or it can be a complex multi-compartment simulation of ion channels behaving according to Hodgkin-Huxley equations where external current is injected as a function of position.

For the analysis proposed in this paper, we simply adopt average firing rates as our encoding model, which we will call $f(x)$. Fitting this model is simple; given some training data, we compute the average firing rate of a cell at every possible position, and those average values are used as the probability of a cell firing at those positions in our held-out test data. This is $P(n|x)$, and it is literally just a matrix. To ensure you understand so far, what should its dimensions be?

Now, Bayesian decoding proposes to estimate what stimulus was presented at each timepoint by using an established encoding model, as well as a prior, to compute $P(x|n)$. This is probability of a stimulus x given the observed neural firing rate. Recall from Bayes theorem that $P(x|n)P(n) = P(n|x)P(x)$. We can therefore solve for $P(x|n)$ with $(P(n|x)P(x))/P(n)$. As long we normalize $P(x|n)$ over all possible $x$, then $P(n)$ does not have to be accounted for. Another way of putting this is that when we fit an encoding model, we are asking what parameters maximize the odds of the observed firing rates given the stimulus, and when we fit our decoding model we are asking what stimulus maximizes the odds of the observed firing rates given some parameters.

Now, if we assume that our observed spikes occur according to a Poisson distribution (meaning our encoding model produces the firing rates that parametrize a Poisson distribution), then for some $\lambda$ (spat out of our encoding model)
$$P(n|x) = \frac{\lambda^ne^{-\lambda}}{n!}$$
Recall that $n$ is just the number of observed spikes. In the case of our firing rate encoding model this becomes
$$P(n|x) = \frac{f(x)^ne^{-f(x)}}{n!}$$
which, when computed across all the cells in our recording, is identical to equation 35 in the linked paper (their τ just puts the firing rates in terms of spikes/second). Then by including the prior $P(x)$, which tells you the probability an animal is at a given position (how would you compute this in your training data?), you obtain equation 36 and solve for $P(x|n)$.
 $$P(x|n) ∝ P(x)\frac{f(x)^ne^{-f(x)}}{n!}$$
 Remember, this $P(x|n)$ is just the probability of being at a position, given the observed neural response, so it serves as a decoder. To decode the animal's position, you then simply choose the most likely position. In other words
 $$\underset{x}{argmax}(P(x|n))$$

Alright, we're finally ready for the project. You have just developed this Bayesian decoding method, but you want to make sure it's working right. Complete the following steps in your own set of python tools. Feel free to implement it in a functional, object-oriented, or interactive way (but think a little about what would make the most sense).



1.   Simulate some synthetic spiking data from a stimulus according to a known encoding model that will serve as the ground truth. Pretend we have a visual stimulus that moves in a perfect circle, where at each timepoint it goes up by one degree of visual angle. Have it move in a full circle twice (so 0-360 degrees twice). Create a set of firing rates for 360 cells, where each cell fires 1 spike/s if the stimulus is in its preferred angle, 1 cell per visual angle. Pass these firing rates through a Poisson distribution (you can use np.random.Generator.poisson()) to obtain your "recorded activity."
2.   Compute your encoding model ($P(n|x)$) as the average firing rate of each cell for every possible stimulus value in half your data. Then in the other half of the data, perform Bayesian decoding using the above equations. Make a plot showing both the true stimulus values, as well as the likelihood distribution at each timepoint. We should be able to see the decoded values track the true ones.
3.   Play around with the number of neurons you use, adding noise to your data, and the number of samples (timepoints) you have. How does your decoding performance vary?
4.   Simulate another dataset, now by creating a 2D arena and a set of cells with 2D gaussian place fields at random locations within this arena (don't ensure that every location is covered by a field, make it random). Simulate a random walk through this arena and save the resulting firing rates as your "recorded activity" (after passing it through a Poisson ditribution). Ensure your gaussian place fields have a maximum value of 20, and therefore your maximum firing rate should be 20.
5.   Repeat the steps in "2" but with this new dataset. Train your encoding model on half your data, now selected randomly from throughout the session (neurons can be nonstationary over sessions!). Hopefully if you wrote your code generally enough, you can reuse what you've already written. Create a plot showing the random walk over time, as well as the likelihoods for each position. We should be able to see the decoded values track the true ones.
6.   When your random walk explores regions of space for which no neurons have place fields, how does the decoder behave? Try adjusting the values for maximum firing rates. How does this change the decoder's performance? Play around with the size of the guassian place fields to increase/decrease overlap between them.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pytoolsJS.plotting import load_style_sheet
load_style_sheet()

np.random.seed(42)

style loaded


In [18]:
def build_clock_stim():
    stimulus = np.zeros((2, 360))
    for deg in range(360):
        x = np.cos(np.radians(deg))
        y = np.sin(np.radians(deg))
        stimulus[:, deg] = [x, y]
    return stimulus

def get_response():
    neural_responses = np.diag([np.random.poisson(1) for i in range(360)])
    return neural_responses

In [20]:
train_responses = get_response()
test_responses = get_response()

In [ ]:
P_x = 1 / 360

answers = np.zeros(360)
for j, x in enumerate(range(360)):
    product = np.prod([train_responses[i, x] ** test_responses[i, x] for i in range(360)])
    summation = np.exp(-1 * np.sum([train_responses[i, x] for i in range(360)]))
    answers[j] = P_x * product * summation
